In [1]:
#! pip3 install ipympl --user

In [1]:
%matplotlib
import numpy as np
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets
from ipywidgets import FloatSlider as Slider
from ipywidgets import IntSlider as Slider_int
from ipywidgets import interact, Layout
from ipywidgets import interactive
import sys
import datetime

Using matplotlib backend: Qt5Agg


In [4]:
os.chdir(r'C:\aa_sem_i\PiPG_I\temat_5\dane_wejsciowe') # Zmiana obszaru roboczego
img = plt.imread('_earth.jpg') # Wczytanie obrazu Ziemi

# Stałe systemu GRS-80
GM =  3986005*10**8
om_e = 7292115*10**-11

def plot_(fi,la, interval):
    ''' Funkcja rysująca wykres'''
    
    markersize = 3 if interval <7 else 6
    
    plt.figure(figsize=(10,5))
    plt.title('Wpływ parametrów orbity na trajektorię satelity',fontsize = 18, pad = 20)
    plt.xlabel('$\lambda_0$',fontsize = 15)
    plt.ylabel('$\phi_0$', fontsize = 15)
    #skalowanie tla
    plt.imshow(img, extent=(-180,180,-90,90))
    #trajektoria satelity i miejscowosc
    plt.plot(la,fi, 'ok', label = f'lokalizacja satelity, interwał {interval} minut', markersize = markersize)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [5]:
# obliczenia
def data(a, e, i, M_0, Om_0, om, T, interval):
    # a - półoś wielka
    # e - mimośród
    # i - nachylenie orbity
    # om - argument perigeum
    # OM_0 - długość węzła wstępującego
    # T - moment czasu
    # interval - interwał rejestracji

    ro = np.pi/180
    i *= ro
    M_0 *= ro
    Om_0 *= ro
    om *= ro
    
    dt = np.arange(0,3600*T+1,60*interval)
    
    n = np.sqrt(GM / a ** 3) # ruch satelity
    M = M_0 + n * dt # anomalia średnia

    def obl_E(E, E0, M, e):
        while abs(E - E0) > 10 ** -3:
            E0 = E
            E = M + e * np.sin(E)
        return E
    
    E = np.array([obl_E(0, 0.2, M_P, e) for M_P in M]) # anomalia mimośrodowa
    
    v = np.arctan2(np.sqrt(1 - e ** 2) * np.sin(E), np.cos(E) - e) # anomalia prawdziwa
    u = v + om # azrgument szerokości
    r = a * (1 - e * np.cos(E)) # promień wodzący satelity
    
    # Współrzędne satelity w płaszczyźnie płaskiej orbity
    xp = r * np.cos(u)
    yp = r * np.sin(u)
    
    OM = Om_0 - om_e * dt # rektascenzja węzła zstępującego
    
    # Współrzędne satelity w układzie geocentrycznym
    x = xp * np.cos(OM) - yp * np.cos(i) * np.sin(OM)
    y = xp * np.sin(OM) + yp * np.cos(i) * np.cos(OM)
    z = yp * np.sin(i)

    # przeliczenie współrzędnych x, y, z satelity na ϕ, λ punktu podsatelitarnego
    ro = 180 / np.pi
    p = np.sqrt(x * x + y * y)
    fi = ro * np.arctan2(z, p)
    la = ro * np.arctan2(y, x)
    
    
    plot_(fi,la,interval) # rysowanie wykresu

In [6]:

# definicja suwaków do wartości parametrów wejściowych
lay = Layout(width='80%', height='15px')
slider_a = Slider_int(value=26555000, min=6400000, max=51200000, step=5, description='$a [m]$', layout=lay)
slider_e = Slider(value=0.072, min=0, max=1, step=0.000000001, description='$e [1]$', layout=lay)
slider_i = Slider(value=63.4, min=0, max=180, step=0.1, description='$i [&deg]$', layout=lay)
slider_M0 = Slider(value=60, min=0, max=360, step=0.1, description='$M_0 [&deg]$', layout=lay)
slider_OMEGA0 = Slider(value=90, min=0, max=360, step=0.1, description='$\Omega_0 [&deg]$', layout=lay)
slider_omega = Slider(value=90, min=0, max=360, step=0.1, description='$\omega [&deg]$', layout=lay)
slider_T = Slider_int(value=24, min=1, max=168, step=1, description='$T [s]$', layout=lay)
slider_interval = Slider_int(value=15, min=1, max=60, step=1, description='$\Delta T [s]$', layout=lay)


# wywołanie funkcji z paraametrami ustawionymi na suwakach
i = interact(data, a=slider_a, e=slider_e, i=slider_i, M_0=slider_M0, Om_0=slider_OMEGA0, om=slider_omega, T = slider_T,
         interval=slider_interval)

interactive(children=(IntSlider(value=26555000, description='$a [m]$', layout=Layout(height='15px', width='80%…

In [7]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')

u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x = 6371 * np.outer(np.cos(u), np.sin(v))
y = 6371 * np.outer(np.sin(u), np.sin(v))
z = 6356 * np.outer(np.ones(np.size(u)), np.cos(v))
# Plot the surface
ax.plot_surface(x, y, z, cmap = 'inferno')


In [34]:
%magic